In [ ]:
from importlib import reload
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, project_root)

import pacing_auction.data as data
import pacing_auction.auction as auction
import pacing_auction.elimination as elimination
import pacing_auction.generator as generator
reload(data)
reload(elimination)
reload(auction)
reload(generator)

sns.set_theme(style="whitegrid")

plt.rcParams.update({
    'font.size': 12,           # Default font size
    'axes.titlesize': 16,      # Title font size
    'axes.labelsize': 14,      # Axis label font size
    'xtick.labelsize': 12,     # X-axis tick label size
    'ytick.labelsize': 12,     # Y-axis tick label size
    'legend.fontsize': 12,     # Legend font size
})

plt.rcParams['figure.dpi'] = 200

OUTPUT_PATH = "/Users/khalid/Desktop/honours-project-writeup/figures/"


In [ ]:
save = True

## Stuff Without Data

In [ ]:
def sample(delta, sigma, mu=0.5) -> float:
    comp_mu = mu - delta if np.random.rand() < 0.5 else mu + delta
    a, b = (0 - comp_mu) / sigma, (1 - comp_mu) / sigma
    return stats.truncnorm.rvs(a, b, loc=comp_mu, scale=sigma) # type: ignore

# Parameters
delta_values = [0, 0.25, 0.5]
sigma = 0.1
n_samples = 100 if not save else 50000
samples = [list[float]() for _ in delta_values]


In [ ]:
# Plot histograms
for i, delta in enumerate(delta_values):
    if len(samples[i]) < n_samples:
        samples[i] = [sample(delta, sigma) for _ in range(n_samples)]

fig, axes = plt.subplots(1, len(delta_values), figsize=(15, 5), sharey=True)
for i, (ax, delta) in enumerate(zip(axes, delta_values)):
    sns.histplot(samples[i], bins=50, kde=True, ax=ax)
    ax.set_title(f"δ = {delta:.2f}")
    ax.set_xlim(0, 1)

axes[0].set_ylabel("Density")
plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + "mixed_gaussian_samples.png")
plt.show()

In [ ]:
n, m, q = 5, 5, 1000
sim = auction.Auction(n, m, q, rng=np.random.default_rng(0))
bids = sim.bids()
bidder = 1
utils = []
for alpha in range(q + 1):
    x, p = sim.auction(adjustment=(bidder, alpha))
    utils.append(sim.utility(x, p)[bidder])

plt.figure(figsize=(10, 5))
plt.title(f"Utility of bidder 1 for different values of alpha_q")
plt.xlabel("alpha_q")
plt.ylabel("utility")
sns.lineplot(x=range(len(utils)), y=utils)
plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + "utility_alpha_q.png")
plt.show()


In [ ]:
plt.figure(figsize=(16, 10))
a = auction.Auction(10, 10, 1000, rng=np.random.default_rng(234))
res = a.responses()
print(res)
utility = res.stats["utility"]
plt.title("Utility of Bidders After Every Best-Response")
plt.xlabel("Iteration")
plt.ylabel("Utility")

for i in range(n):
    utility_i = utility[i]
    sns.lineplot(x=range(len(utility_i)), y=utility_i, label=f"Bidder {i + 1}")

plt.legend()
plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + "utility_best_response.png")
plt.show()

## Loading in Data


In [ ]:
dfs = []
for i in range(1, 5 + 1):
    df = pd.read_csv(f"../results/v4.{i}/data.csv")
    df["run"] = i
    dfs.append(df)

dfs.append(pd.read_csv("../results/v4.1-binary/data.csv"))
dfs.append(pd.read_csv("../results/v4.2-binary/data.csv"))

df = pd.concat(dfs, ignore_index=True)
df["index"] = df.index
display(df.shape)
display(df.info())
display(df.head())

In [ ]:
df['time_per_iteration'] = df['runtime'] / df['iterations']
df_time_per_iteration_pivot = df.pivot_table(
    index='n', columns='m', values='time_per_iteration', aggfunc='median'
)

plt.figure(figsize=(10, 8))
sns.heatmap(df_time_per_iteration_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Time per Iteration (s)'})
plt.title('Heatmap of Time per Iteration by n and m')
plt.xlabel('m')
plt.ylabel('n')
if save:
    plt.savefig(OUTPUT_PATH + "time_per_iteration_heatmap.png")
plt.show()

In [ ]:
complete = df[df["generator"] == "complete"]
sampled = df[df["generator"] == "sampled"]
correlated = df[df["generator"] == "correlated"]
binary = df[df["generator"] == "binary"]

elim_all = df[df["elim_strategy"] == "all"]
elim_subsequent = df[df["elim_strategy"] == "subsequent"]
elim_current = df[df["elim_strategy"] == "current"]

print(complete.shape)
print(sampled.shape)
print(correlated.shape)
print(binary.shape)

print(elim_all.shape)
print(elim_subsequent.shape)
print(elim_current.shape)

In [ ]:
from matplotlib.colors import Normalize

# Create pivot table for binary data

complete_pivot = complete.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
sampled_pivot = sampled.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
correlated_pivot = correlated.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
binary_pivot = binary.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
# Create a figure with a 2x2 grid layout for four heatmaps and one colorbar
fig = plt.figure(figsize=(20, 18))

# Create a grid layout with space for the colorbar
grid_spec = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.1], height_ratios=[1, 1])

# Create four subplots for the heatmaps
ax1 = fig.add_subplot(grid_spec[0, 0])  # Top left
ax2 = fig.add_subplot(grid_spec[0, 1])  # Top right
ax3 = fig.add_subplot(grid_spec[1, 0])  # Bottom left
ax4 = fig.add_subplot(grid_spec[1, 1])  # Bottom right
cbar_ax = fig.add_subplot(grid_spec[:, 2])  # Right side, spanning both rows

# Find the global min and max values across all four datasets for consistent color scale
vmin = min(complete_pivot.min().min(), sampled_pivot.min().min(),
           correlated_pivot.min().min(), binary_pivot.min().min())
vmax = max(complete_pivot.max().max(), sampled_pivot.max().max(),
           correlated_pivot.max().max(), binary_pivot.max().max())
norm = plt.Normalize(vmin=vmin, vmax=vmax)

# Plot the first heatmap (Complete) in top left
sns.heatmap(complete_pivot, annot=True, cmap="YlGnBu", fmt='.1%',
            cbar=False, ax=ax1, norm=norm)
ax1.set_title('PNE Frequency (Complete)')
ax1.set_xlabel('m')
ax1.set_ylabel('n')

# Plot the second heatmap (Sampled) in top right
sns.heatmap(sampled_pivot, annot=True, cmap="YlGnBu", fmt='.1%',
            cbar=False, ax=ax2, norm=norm)
ax2.set_title('PNE Frequency (Sampled)')
ax2.set_xlabel('m')
ax2.set_ylabel('n')

# Plot the third heatmap (Correlated) in bottom left
sns.heatmap(correlated_pivot, annot=True, cmap="YlGnBu", fmt='.1%',
            cbar=False, ax=ax3, norm=norm)
ax3.set_title('PNE Frequency (Correlated)')
ax3.set_xlabel('m')
ax3.set_ylabel('n')

# Plot the fourth heatmap (Binary) in bottom right
sns.heatmap(binary_pivot, annot=True, cmap="YlGnBu", fmt='.1%',
            cbar=False, ax=ax4, norm=norm)
ax4.set_title('PNE Frequency (Binary)')
ax4.set_xlabel('m')
ax4.set_ylabel('n')

# Create a colorbar in the dedicated axis
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap="YlGnBu"),
                   cax=cbar_ax)
cbar.set_label('PNE Frequency')

# Add a main title for the entire figure
fig.suptitle('PNE Frequency by Method', fontsize=16, y=0.98)

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave room for the suptitle

if save:
    plt.savefig(OUTPUT_PATH + "heatmap_pne_frequency_grid.png")

# Show the figure with all four heatmaps in a 2x2 grid with shared colorbar
plt.show()

In [ ]:
# Create pivot tables for the number of iterations
complete_iterations_pivot = complete.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
sampled_iterations_pivot = sampled.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
correlated_iterations_pivot = correlated.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
binary_iterations_pivot = binary.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)

# Create a figure with a 2x2 grid layout
fig = plt.figure(figsize=(20, 18))

# Create a grid layout with space for the colorbar
grid_spec = fig.add_gridspec(2, 3, width_ratios=[1, 1, 0.1], height_ratios=[1, 1])

# Create four subplots for the heatmaps
ax1 = fig.add_subplot(grid_spec[0, 0])  # Top left
ax2 = fig.add_subplot(grid_spec[0, 1])  # Top right
ax3 = fig.add_subplot(grid_spec[1, 0])  # Bottom left
ax4 = fig.add_subplot(grid_spec[1, 1])  # Bottom right
cbar_ax = fig.add_subplot(grid_spec[:, 2])  # Right side, spanning both rows

# Find the global min and max values across all four datasets for consistent color scale
vmin = min(complete_iterations_pivot.min().min(), sampled_iterations_pivot.min().min(),
           correlated_iterations_pivot.min().min(), binary_iterations_pivot.min().min())
vmax = max(complete_iterations_pivot.max().max(), sampled_iterations_pivot.max().max(),
           correlated_iterations_pivot.max().max(), binary_iterations_pivot.max().max())
norm = plt.Normalize(vmin=vmin, vmax=vmax)

# Plot the first heatmap (Complete) in top left
sns.heatmap(complete_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax1, norm=norm)
ax1.set_title('Median Iterations (Complete)')
ax1.set_xlabel('m')
ax1.set_ylabel('n')

# Plot the second heatmap (Sampled) in top right
sns.heatmap(sampled_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax2, norm=norm)
ax2.set_title('Median Iterations (Sampled)')
ax2.set_xlabel('m')
ax2.set_ylabel('n')

# Plot the third heatmap (Correlated) in bottom left
sns.heatmap(correlated_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax3, norm=norm)
ax3.set_title('Median Iterations (Correlated)')
ax3.set_xlabel('m')
ax3.set_ylabel('n')

# Plot the fourth heatmap (Binary) in bottom right
sns.heatmap(binary_iterations_pivot, annot=True, cmap="YlGnBu",
            cbar=False, ax=ax4, norm=norm)
ax4.set_title('Median Iterations (Binary)')
ax4.set_xlabel('m')
ax4.set_ylabel('n')

# Create a colorbar in the dedicated axis
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap="YlGnBu"),
                   cax=cbar_ax)
cbar.set_label('Median Iterations')

# Add a main title for the entire figure
fig.suptitle('Median Iterations by Method', fontsize=16, y=0.98)

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave room for the suptitle

if save:
    plt.savefig(OUTPUT_PATH + "heatmap_iterations_grid.png")

# Show the figure with all four heatmaps in a 2x2 grid with shared colorbar
plt.show()

In [ ]:
# Define the datasets
datasets = [
    ('Complete', complete),
    ('Sampled', sampled),
    ('Correlated', correlated),
    ('Binary', binary)
]

# Create a single large figure with a 4x2 grid (4 rows, 2 columns)
fig, axes = plt.subplots(4, 2, figsize=(16, 20))

# Loop through each dataset and create plots
for i, (name, dataset) in enumerate(datasets):
    # Filter for PNEs and Cycles in this dataset
    pnes = dataset[dataset['result_type'] == 'PNE']
    cycles = dataset[dataset['result_type'] == 'Cycle']

    # Calculate percentage of PNEs and Cycles
    pne_percent = round(pnes.shape[0] * 100 / dataset.shape[0], 2)
    cycle_percent = round(100 - pne_percent, 2)

    # Get the current row's axes
    ax1 = axes[i, 0]  # Left plot (PNEs)
    ax2 = axes[i, 1]  # Right plot (Cycles)

    # First subplot for PNEs
    sns.histplot(
        pnes[pnes['iterations'] < 1000]['iterations'],  # type: ignore
        bins=100,
        kde=True,
        ax=ax1
    )
    ax1.set_xlim(0, 1000)
    ax1.set_title(f'Iteration Count for PNEs ({name}): {pne_percent}%')
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Frequency')

    # Second subplot for Cycles
    sns.histplot(
        cycles[cycles['iterations'] < 1000]['iterations'],  # type: ignore
        bins=100,
        kde=True,
        ax=ax2
    )
    ax2.set_xlim(0, 1000)
    ax2.set_title(f'Iteration Count for Cycles ({name}): {cycle_percent}%')
    ax2.set_xlabel('Iterations')
    ax2.set_ylabel('Frequency')

    # Get the maximum y-limit from both subplots and set them equal
    max_ylim = max(ax1.get_ylim()[1], ax2.get_ylim()[1])
    ax1.set_ylim(0, max_ylim)
    ax2.set_ylim(0, max_ylim)

# Add a super title for the entire figure
plt.suptitle('Iteration Count Distributions - All Methods', fontsize=20, y=0.98)

# Adjust layout
plt.tight_layout(rect=(0, 0, 1, 0.97))

# Save the combined figure
if save:
    plt.savefig(OUTPUT_PATH + "iteration_count_all_methods.png", dpi=300)

plt.show()

In [ ]:
restricted = df[(df["iterations"] < 1000) & (df["generator"] != "binary")]
# Filter out iterations above 1000 to keep the plot readable
plt.figure(figsize=(16, 10))

# Plot histograms for different values of n (keeping m constant)
plt.subplot(1, 2, 1)
for n_val in [2, 4, 6, 8, 10]:
    subset = restricted[restricted['n'] == n_val]
    if not subset.empty:
        sns.kdeplot(subset['iterations'], label=f'n={n_val}', fill=True, alpha=0.2) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution by n Values')
plt.xlabel('Iterations')
plt.ylabel('Frequency')
plt.legend()

# Plot histograms for different values of m (keeping n constant)
plt.subplot(1, 2, 2)
for m_val in [2, 4, 6, 8, 10]:
    subset = restricted[restricted['m'] == m_val]
    if not subset.empty:
        sns.kdeplot(subset['iterations'], label=f'm={m_val}', fill=True, alpha=0.2) # type: ignore
plt.xlim(0, 1000)
plt.title('Iteration Count Distribution by m Values')
plt.xlabel('Iterations')
plt.ylabel('Frequency')
plt.legend()

# Get the maximum y-limit from both subplots
ax1 = plt.gcf().axes[0]
ax2 = plt.gcf().axes[1]
y_max = max(ax1.get_ylim()[1], ax2.get_ylim()[1])

# Set the same y-limit for both subplots
ax1.set_ylim(0, y_max)
ax2.set_ylim(0, y_max)

plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + "iteration_count_by_n_m.png")
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter
# Your existing code up until the plotting part...

plt.figure(figsize=(16, 10))

# Calculate percentage of PNEs for each sigma-delta combination
sigma_delta_pne = []
for (sigma, delta), df_group in correlated.groupby(["sigma", "delta"]):
    pnes_group = df_group[df_group['result_type'] == 'PNE']
    pne_percentage = (pnes_group.shape[0] / df_group.shape[0]) * 100

    sigma_delta_pne.append({'sigma': sigma, 'delta': delta, 'pne_percentage': pne_percentage})

# Convert to DataFrame
df_sigma_delta = pd.DataFrame(sigma_delta_pne)


# Get unique sigma values
sigma_values = sorted(df_sigma_delta['sigma'].unique())

# Plot a line for each sigma value
for sigma in sigma_values:
    sigma_data = df_sigma_delta[df_sigma_delta['sigma'] == sigma]
    sigma_data = sigma_data.sort_values('delta')
    plt.plot(sigma_data['delta'], sigma_data['pne_percentage'], marker='o', linewidth=2, label=f'σ = {sigma}')

# Format y-axis to show percentage
def percentage_formatter(x, p):
    return f'{x:.0f}%'

plt.gca().yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

plt.title('Percentage of PNEs by Delta (δ) for Each Sigma (σ) Value', fontsize=16)
plt.xlabel('Delta (δ)', fontsize=14)
plt.ylabel('Percentage of PNEs (%)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Sigma Values', fontsize=12)
plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + "pne_percentage_by_sigma_delta.png")
plt.show()

In [ ]:
# Load matrix files from results directories
stats = [
    np.load(f"../results/v4.{i}/auction_matrices.npz", allow_pickle=True)
    for i in range(1, 5 + 1)
]

stats.append(np.load(f"../results/v4.1-binary/auction_matrices.npz", allow_pickle=True))
stats.append(np.load(f"../results/v4.2-binary/auction_matrices.npz", allow_pickle=True))


In [ ]:
# Create two sets of pacing multipliers
pne_alpha_qs_complete = []
pne_alpha_qs_other = []
experiment_index = 0

for matrix_data in stats:
    alpha_q_vectors = matrix_data['alpha_q_vectors']
    for auction_index in range(alpha_q_vectors.shape[0]):
        experiment_row = df.iloc[experiment_index]
        if experiment_row["result_type"] == "PNE" and experiment_row["elim_strategy"] == "subsequent":
            if experiment_row["generator"] == "complete":
                pne_alpha_qs_complete.extend(alpha_q_vectors[auction_index])
            elif experiment_row["generator"] != "complete":
                pne_alpha_qs_other.extend(alpha_q_vectors[auction_index])
        experiment_index += 1

print(f"Total equilibrium pacing multipliers collected (Complete): {len(pne_alpha_qs_complete)}")
print(f"Total equilibrium pacing multipliers collected (Correlated and Sampled): {len(pne_alpha_qs_other)}")

# Create a single figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot the Complete data on the left subplot
sns.histplot(pne_alpha_qs_complete, bins=100, ax=ax1)
ax1.set_xlabel('Equilibrium Pacing Multiplier (α) Values')
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Equilibrium Pacing Multipliers\nin PNE (Complete)')

# Plot the Correlated and Sampled data on the right subplot
sns.histplot(pne_alpha_qs_other, bins=100, ax=ax2)
ax2.set_xlabel('Equilibrium Pacing Multiplier (α) Values')
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Distribution of Equilibrium Pacing Multipliers\nin PNE (Correlated and Sampled)')

# Make the layout tight
plt.tight_layout()

# Save the combined figure
if save:
    plt.savefig(OUTPUT_PATH + 'pne_alpha_dist.png', dpi=300)

# Display the figure
plt.show()

In [ ]:
# Extract equilibrium pacing multipliers for binary dataset only
from collections import Counter


binary_pne_alpha_qs = []
experiment_index = 0

for matrix_data in stats:
    alpha_q_vectors = matrix_data['alpha_q_vectors']
    print(len(alpha_q_vectors))
    for auction_index in range(alpha_q_vectors.shape[0]):
        experiment_row = df.iloc[experiment_index]
        if (experiment_row["result_type"] == "PNE" and experiment_row["generator"] == "binary"):
            binary_pne_alpha_qs.extend(alpha_q_vectors[auction_index])
        experiment_index += 1

print(Counter(binary_pne_alpha_qs))

print(f"Total equilibrium pacing multipliers collected from binary: {len(binary_pne_alpha_qs)}")

# Create histogram with styling
plt.figure(figsize=(10, 6))
sns.histplot(binary_pne_alpha_qs, bins=100)

# Add labels and title
plt.xlabel('Equilibrium Pacing Multiplier (α) Values', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Equilibrium Pacing Multipliers in PNE (Binary)')
plt.grid(axis='y', alpha=0.3)


plt.tight_layout()

if save:
    plt.savefig(OUTPUT_PATH + 'pne_alpha_dist_binary.png', dpi=300)

plt.show()

In [ ]:
# Load matrix files from results directories
stats = [
    np.load(f"../results/v4.{i}/auction_stats.npz", allow_pickle=True)
    for i in range(1, 5 + 1)
]

stats.append(np.load(f"../results/v4.1-binary/auction_stats.npz", allow_pickle=True))
stats.append(np.load(f"../results/v4.2-binary/auction_stats.npz", allow_pickle=True))
print(stats)

In [ ]:
pne_welfare = []
experiment_index = 0

for matrix_data in stats:
    lw_stats = matrix_data['social_welfare_stats']
    for auction_index in range(lw_stats.shape[0]):
        experiment_row = df.iloc[experiment_index]
        if (experiment_row["result_type"] == "PNE" and experiment_row["generator"] != "binary"):
            pne_welfare.append(lw_stats[auction_index])
        experiment_index += 1

In [ ]:
# max_welfare = [max(p) for p in pne_welfare]
# last_welafare = [p[-1] for p in pne_welfare]
ratio = np.array([max(p) / p[-1]  for p in pne_welfare if max(p) / p[-1] < 1.5])
# summary stats of ratio
print(f"Mean: {np.mean(ratio)}")
print(f"Median: {np.median(ratio)}")
print(f"Std: {np.std(ratio)}")
print(f"Min: {np.min(ratio)}")
print(f"Max: {np.max(ratio)}")
print(f"Efficient: ", len(ratio[ratio == 1]) / len(ratio))

plt.figure(figsize=(10, 6))
sns.histplot(ratio, bins=100)
plt.xlabel('Liquid Welfare Values', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Liquid Welfare in PNE')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + 'pne_liquid_welfare_dist.png', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.title("Liquid Welfare Over Iterations")
plt.xlabel("Iterations")
plt.ylabel("Liquid Welfare")
sns.lineplot(pne_welfare[232])

plt.tight_layout()
if save:
    plt.savefig(OUTPUT_PATH + 'pne_liquid_welfare_over_iterations.png', dpi=300)
plt.show()